#The action plan on MNIST dataset

### 1. Prepare our data and preprocess it.
### 2. Create training, validation and test dataset
### 3. Outline the model and choose the activation functions
### 4. Set the appropriate advanced optimizers and loss funtions
### 5. Make it learn
### 6. Test the accuracy of the model

# Importing the relevant packages

In [31]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfdf

##Storing the data as the mnist_dataset variable and setting the arguments ***as_supervised*** and ***with_info*** to **True** so that the data loads into two tuple structure **[input, target]** and provides a tuple containing info about version, features, number of samples of the dataset.

In [32]:
mnist_dataset, mnist_info = tfdf.load(name='mnist', with_info=True, as_supervised=True)


#Prepare training, validation and test datasets and preprocess it.

In [33]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*(mnist_info.splits['train'].num_examples)
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


##Scaling the data (inputs between 0 and 1) by defining a function named **scale**.
### since, MNIST images contain values from 0 to 255 on the grey scale therefore if we devide each element by 255, we will get the desired result.

In [34]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label


In [35]:
scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

### Since we will be batching , we'de better shuffle the data. it should be as randomly spread as possible so that the batching works as intended.
##here, 1 < BUFFER_SIZE < num_samples

In [36]:
BUFFER_SIZE = 1000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)



# I will be using Minibatch gradient descent to the model.
## 1 < BATCH_SIZE < number of samples = mini-batch GD
### Note: Model expects our validation set in batch form too. we choose the batch size for validation set equal to the total number of validation samples so we will have a single batch for this.
### since, during validating and testing we want the exact values.  

In [37]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

### Our data is iterable and in 2-tuple format (as_supervised=True)
###therefore, first extract and convert the validation inputs and targets appropriately.

In [38]:
validation_inputs, validation_targets = next(iter(validation_data))

#Outline the model and choose the activation functions
## Since, our data is (from tfdf) is such that each input is 28 * 28 * 1 or a tensor of rank 3 so we need to flatten the images(tensor) into a vector.
##

In [65]:

input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])


# Choosing Optimizer and the Loss function
##Here, model.compile(optimizer, loss , metrics) configures the model for training.

In [66]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model

In [67]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 7s - 14ms/step - accuracy: 0.9071 - loss: 0.3199 - val_accuracy: 0.9530 - val_loss: 0.1567
Epoch 2/5
540/540 - 6s - 11ms/step - accuracy: 0.9616 - loss: 0.1265 - val_accuracy: 0.9653 - val_loss: 0.1165
Epoch 3/5
540/540 - 6s - 11ms/step - accuracy: 0.9728 - loss: 0.0893 - val_accuracy: 0.9697 - val_loss: 0.1067
Epoch 4/5
540/540 - 10s - 18ms/step - accuracy: 0.9801 - loss: 0.0656 - val_accuracy: 0.9732 - val_loss: 0.0935
Epoch 5/5
540/540 - 10s - 18ms/step - accuracy: 0.9829 - loss: 0.0555 - val_accuracy: 0.9737 - val_loss: 0.0938


##We usually look at the validation loss (or set early stopping mechanisms) to determine whether the model is overfitting.
##Validation_accuracy = True accuracy of the model for the epoch
##Since, training accuracy is the average accuracy across batches while the validation accuracy is that of the whole validation set.

#To assess the averall accuracy of our model we look at the validation accuracy for the last epoch.

#Train:-     (This make our model to learn and optimizes the model)

#Validate:-    (This make sure our parameters- the weights and biases- don't overfit)

#Test:-    (This make sure our Hyperparameters- width, depth, batch size, number of epochs, etc.- don't overfit)

In [72]:
test_accuracy, test_loss = model.evaluate(test_data)            #model.evaluate returns the loss value and matrics values for the model in the test mode
print('Test accuracy: {0: .2f}% Test loss: {1: .2f}%'.format(test_accuracy*100, test_loss*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 830ms/step - accuracy: 0.9734 - loss: 0.0845
Test accuracy:  8.45% Test loss:  97.34%


#Getting a test accuracy very close to the validation accuracy shows that we have not overfit over model
#Therefore, the model works with 97.34% accuracy in real life deployment.